In [18]:
import numpy as np
import tqdm.notebook as tqdm

In [19]:
list_file = []

with open('input.txt','r') as input_file:
    for nums in input_file:
        nums_split = nums.split()
        list_file.append(nums_split)

In [20]:
task = int(list_file[0][0])
category_task = np.array(list_file[1], int)
time_for_tusk = np.array(list_file[2], float)
num_devs = int(list_file[3][0])
coef_devs = np.array(list_file[4::], float)

size_population = 2000
max_len_individual = 1000
size_selection = 700
p_mutation_ind= 0.2
p_mutation_gen=0.1

In [21]:
class GeneticAlgorithm:
    
    def __init__(
        self,
        task,
        category_task ,
        time_for_tusk ,
        num_devs,
        coef_devs,
        size_population,
        max_len_individual,
        size_selection,
        p_mutation_ind,
        p_mutation_gen
    ):
        self.task = task
        self.category_task = category_task
        self.time_for_tusk = time_for_tusk
        self.num_devs = num_devs
        self.coef_devs = coef_devs
        self.size_population = size_population
        self.max_len_individual = max_len_individual
        self.size_selection = size_selection
        self.p_mutation_ind = p_mutation_ind
        self.p_mutation_gen = p_mutation_gen

        self.rng = np.random.default_rng()
        self.population = self.rng.integers(low=0, high=10, size=(size_population, max_len_individual))

    def fitness(self) :
        temp = coef_devs[self.population, self.category_task-1] * self.time_for_tusk

        list_of_time = []

        for dev in range(len(coef_devs)):
            list_of_time.append(temp.sum(axis=1, where = self.population == dev))

        array_of_time = np.array(list_of_time)
        
        max_time_dev = array_of_time.max(axis = 0) 
        return 100/max_time_dev
    
    def selection(self):
        self.selected_population = self.population[np.argsort(self.fitness())[-self.size_selection:]][::-1]
    
    def crossover(self):
        n_children = self.size_population - self.size_selection
        
        idx1  = self.rng.integers(low=0, high=self.size_selection,     size=n_children)
        shift = self.rng.integers(low=1, high=self.size_selection - 1, size=n_children)
        idx2  = (idx1 + shift) % self.size_selection
        
        self.slice_point = self.rng.integers(low=1, high=self.max_len_individual - 1, size=n_children)
        self.new_generation = np.where(
            np.arange(self.max_len_individual)[np.newaxis] < self.slice_point[:, np.newaxis],
            self.selected_population[idx1],
            self.selected_population[idx2]
        )
        
    def mutation(self):
        self.mask = self.rng.random(size=self.new_generation.shape[0]) <= self.p_mutation_ind
        self.mutating_children = self.new_generation[self.mask]
        self.mutating_children[...] = np.where(
            self.rng.random(size=self.mutating_children.shape) <= self.p_mutation_gen,
            self.rng.integers(low=0, high=4, size=self.mutating_children.shape),
            self.mutating_children
        )

            
    def step(self):
        ga.selection()
        ga.crossover()
        ga.mutation()
        self.population = np.concatenate([self.selected_population, self.new_generation, self.mutating_children], axis=0)

In [22]:
ga = GeneticAlgorithm(task,category_task ,time_for_tusk ,num_devs,coef_devs,size_population,max_len_individual,size_selection,p_mutation_ind,p_mutation_gen)

In [23]:
pbar = tqdm.trange(100)
for i in pbar:
    ga.step()
    pbar.set_description(f'{ga.fitness().max()}')

  0%|          | 0/100 [00:00<?, ?it/s]

In [27]:
ga.fitness().argmax()

0

In [28]:
print(*ga.population[0]+1)

3 5 6 5 3 8 7 2 9 7 7 4 10 5 8 9 9 3 9 9 4 4 10 10 5 2 4 8 5 5 6 8 9 2 10 10 7 7 10 2 2 9 9 10 3 10 1 8 2 2 10 6 6 9 6 6 2 8 6 9 3 9 4 5 3 9 5 1 7 5 10 10 6 9 8 10 7 3 6 10 4 8 4 10 2 8 9 6 3 2 10 7 7 4 1 6 6 2 6 1 8 3 2 2 6 8 1 2 1 3 6 8 5 5 8 1 3 2 2 5 8 7 3 9 8 9 2 9 7 9 8 8 4 5 7 6 4 1 7 5 1 6 7 6 7 1 4 6 3 5 3 6 8 1 1 3 7 2 7 6 6 10 1 4 9 8 1 8 10 7 8 3 8 10 4 9 3 2 3 7 7 8 4 10 3 4 6 7 1 2 7 1 6 2 10 3 1 8 3 5 9 4 1 4 7 9 7 2 9 9 5 1 10 10 9 9 6 6 7 3 3 8 10 4 8 3 2 4 6 10 10 8 8 4 6 7 1 1 6 3 3 6 3 10 10 9 8 2 6 3 8 5 7 7 10 10 9 7 6 6 3 7 9 2 4 3 2 9 2 7 6 2 1 9 8 7 7 3 10 1 8 10 1 3 4 9 8 1 8 9 8 1 10 1 10 7 2 5 9 2 6 2 7 5 10 10 4 4 2 1 10 5 9 4 2 3 5 7 10 2 2 10 6 3 3 9 8 7 7 9 10 9 1 6 4 6 1 3 10 3 4 1 4 3 2 5 7 4 2 1 7 9 1 1 10 9 9 1 2 7 8 7 9 3 2 9 6 2 10 6 8 1 7 10 7 5 5 6 1 6 4 4 3 10 7 4 2 4 6 3 1 8 5 9 1 9 6 6 9 5 2 1 3 10 2 1 5 9 5 4 5 10 4 9 6 8 1 6 9 2 7 7 3 4 3 9 10 3 4 6 9 10 10 5 5 6 2 6 2 10 5 1 6 6 4 5 7 6 4 9 3 5 5 7 5 8 7 2 1 5 7 8 8 5 2 9 5 2 8 10 4 1 1 8 4

In [29]:
ga.fitness().max()

0.16299254309115366